In [4]:
FASTA="../annotate/data/FASTA"
NIDO="../annotate/in/Nidovirales.fasta"
PROFILE="./aln/F189.aln.fasta"

m8_file_path = "./F189-search/alnResult.m8"




### 1. parse seq id 

In [5]:
import pandas as pd

# Load the .m8 file into a pandas DataFrame

columns = [
    'query', 'subject', 'percentage_identity', 'alignment_length',
    'mismatches', 'gap_opens', 'query_start', 'query_end', 
    'subject_start', 'subject_end', 'e_value', 'bit_score'
]
m8_df = pd.read_csv(m8_file_path, sep='\t', names=columns)

# Extract unique sequence identifiers
sequence_ids = set(m8_df['query']).union(set(m8_df['subject']))

### Step 2: Fetch sequences from a FASTA file

In [10]:
from Bio import SeqIO

# Load sequences from a FASTA file
sequences:dict = {}
Sources = [PROFILE, NIDO, FASTA]
for fasta_file_path in Sources:
    for record in SeqIO.parse(fasta_file_path, 'fasta'):
        if record.id not in sequences:
            sequences = { **sequences, record.id: record}
#sequences = {record.id: record for record in SeqIO.parse(fasta_file_path, 'fasta')}

# Filter sequences based on the unique identifiers from the .m8 file
filtered_sequences = [sequences[seq_id] for seq_id in sequence_ids if seq_id in sequences]



### Step 3: Write to a multifasta file

In [11]:
# Write the filtered sequences to a multifasta file
output_fasta_file_path = 'F189-search/filtered_sequences.fasta'
with open(output_fasta_file_path, 'w') as output_handle:
    SeqIO.write(filtered_sequences, output_handle, 'fasta')

# _________________________________________________________________________________________________________________

### workflow

1. adding sequence "polymerase" from francois                       **&rarr; NO**
2. retreve fasta seq from all record                                **&rarr; OK**
3. filter - remove redundancy - cluster                             **&rarr; OK**
4. align -> tree                                                    **&rarr; ...**
5. cluster -> network graph (qvge --balloon)
6. short align -> leaders -> Espript
7. locate MOTIF (Shannon et al, 2020) -> cut-off entropy -> MOTIF
8. heatmap , test with all profile sequences